In [ ]:
import json
import numpy as np
import torch
from torch_geometric.data import Data
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from transformers import pipeline


In [2]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [3]:
with open('edinburgh-keywords_train.json', 'r') as f:
    train_data = json.load(f)

keywords = list(train_data['np2count'].keys())

keyword_set = set(keywords)

In [4]:
def extract_users(info):
    l_user, user2kw = [], []
    for ii in info:
        lus = info[ii]
        for u in lus:
            if u not in l_user:
                l_user.append(u)
                user2kw.append([])
            idx = l_user.index(u)
            user2kw[idx].append(ii)
    return l_user, user2kw

In [5]:
train_users, train_users2kw = extract_users(train_data['np2users'])


In [8]:
train_users

['GN3wBXDaLLmY6Oitfy3T8g',
 'hqZiOnwZ3J1s7piCBUN6Ow',
 'LnkhyQkyvXJBUMEZGWsAVw',
 'VRVCKQhYDCkzaEDce8GEtQ',
 'SxV1Jq7UANuSYpn42JXvOA',
 'aIJ0eYL3g-lWuUZdRnsmLw',
 'pxspA49jdI2Z9RD_UD1A_A',
 'fKi8ZF3xpJJkgSusRubeMA',
 'PQq5JGWvr0u_IKDXZyDAgA',
 '5iRCT4uAsyP1UoEUoxbylA',
 'gHFoIL1HxfJ7NTV414b0zQ',
 'MWxyukeYIxIbuzpciZNhXw',
 '8rI2CoS4ltQePbZceSOgow',
 'F7o19Xwgh-Qtg-8zgkFNiw',
 'txzOZrHK9EVY0kYjSuXkKg',
 'F9nMHS70LBqEwKYFFuJ_0Q',
 'JS6RV2pxmYQ-56Eb_b1v2A',
 'rl4hqDnywPJDCK4wfbXMzg',
 '6Rb1cZiuhBPILcHxfbInig',
 'nltUy-nDWqKcDHGd12ZfLg',
 '9RGIvzCGHp8ELF_JgAlbZA',
 'DhbExwYDt5zckqXi9wfnuw',
 'O6dMShJ-dIX_4Qsq6mvvqA',
 'rXvsM4vsvrhmLKXu10HRbQ',
 'J6E6NSpJqMBib9ydbchicg',
 'frthTngbBHgSdk4j85HmEQ',
 'neMi0e3dZaUtX7Sg-3Qytw',
 'bVNSTKNTBklYhUhalNSSdQ',
 '_5ujlgjN2bU8H_kKYSbzdw',
 'KPY6FvT6h-r0W0HAC1Buag',
 'Cy1PV2TdYeDFA16mUOlr3Q',
 'U55y_61paae1f6r6vgITJw',
 'yI4LI8-aYYUS9MmrfpidMQ',
 'S7KrrI6hpAMcvCIfMxbXmw',
 'WrMDMbEt9ap0VbmNsnKvKA',
 'H5TEwR-OD0XxILGNaewohA',
 'oKIoEwnyZ5bHSeKLzjcZgQ',
 

In [6]:
restaurant_set = set()
listres = []
for kw in train_data['np2rests'].keys():
    listres.extend(train_data['np2rests'][kw].keys())
restaurant_set = set(listres)

keyword_set = list(keyword_set)
restaurant_set = list(restaurant_set)
restaurants = len(listres)
num_keywords = len(keyword_set)
num_restaurants = len(restaurant_set)
a = np.zeros((num_keywords, num_restaurants))

In [7]:
a.shape

(9798, 937)

In [8]:
for kw in train_data['np2rests'].keys():
    for res in train_data['np2rests'][kw].keys():
        idx_kw = keyword_set.index(kw)
        idx_res = restaurant_set.index(res)
        a[idx_kw][idx_res] = 1

In [9]:
keyword_embeddings = model.encode(list(keyword_set))

In [10]:
with open('edinburgh-keywords_test.json', 'r') as r:
    test_data = json.load(r)

user_keywords = list(test_data['np2reviews'].keys())
user_keywords_list = list(user_keywords)

In [11]:
test_users, test_users2kw = extract_users(test_data['np2users'])


In [12]:
test_keywords = [kw for sublist in test_users2kw for kw in sublist]
test_keyword_embeddings = model.encode(test_keywords)

similarity_scores = cosine_similarity(test_keyword_embeddings, keyword_embeddings)

In [13]:
filtered_keywords = []
for i, user_kw in enumerate(test_users2kw):
    updated_user_kw = []
    for kw in user_kw:
        if kw not in keyword_set:
            test_idx = test_keywords.index(kw)
            sim_scores = similarity_scores[test_idx]

            best_match_idx = np.argmax(sim_scores)
            best_match_keyword = keyword_set[best_match_idx]

            updated_user_kw.append(best_match_keyword)
        else:
            updated_user_kw.append(kw)

    filtered_keywords.append(updated_user_kw)

test_users2kw = filtered_keywords

In [19]:
updated_user_kw

['chicken',
 'royal mile',
 'cocktail',
 'wife',
 'break',
 'naan',
 'salmon',
 'good food',
 'must',
 'cool little place',
 'cocktails',
 'hake',
 'good ingredients',
 'veggie',
 'class',
 'best indian food',
 'scottish cuisine',
 'pot',
 'drawback',
 'english',
 'old school',
 'lunch specials',
 'salad']

In [18]:
filtered_keywords

[['flatmates',
  'food',
  'while',
  'menu',
  'door',
  'week',
  'hall',
  'range',
  'classic dishes',
  'crispy beef',
  'fried rice',
  'bag',
  'prawn crackers',
  'minutes',
  'reality',
  'fiver',
  'beef',
  'good quality',
  'spicy sauce',
  'bit',
  'wait',
  'friend',
  'meal',
  'price',
  'standard',
  'quality',
  'shop',
  'chance',
  'hands',
  'rice',
  'flavours',
  'decor',
  'restaurant',
  'part',
  'soup',
  'size',
  'way',
  'dishes',
  'taste',
  'edinburgh',
  'tea',
  'place',
  'weekend',
  'table',
  'day',
  'need',
  'guy',
  'sign',
  'lot',
  'head',
  'selection',
  'group',
  'tomato',
  'chicken',
  'pastries',
  'main course',
  'dad',
  'eye',
  'service',
  'fish',
  'option',
  'variety',
  'sauce',
  'fish',
  'taiwan',
  'name',
  'great food',
  'prices',
  'party',
  'speed',
  'general atmosphere',
  'excellent fish',
  'style',
  'chips',
  'haddock',
  'counter',
  'hour',
  'crispy chips',
  'can',
  'coke',
  'portion',
  'fish',
  'ch

In [14]:
results = []
for kw in test_users2kw:
    t = np.zeros((1, len(keyword_set)))
    keywords = kw[:10]
    for keys in keywords:
        if keys in keyword_set:
            idx_kw = keyword_set.index(keys)
            t[0][idx_kw] = 1
    R = np.dot(t, a)
    result = np.argsort(R[0])[::-1][:10]
    results.append(result)

In [15]:
if __name__ == "__main__":
        for i, result in enumerate(results):
            restaurant_names = [restaurant_set[idx] for idx in result]
            print(f"The result for user {i} is: {restaurant_names}")

The result for user 0 is: ['xbCNlp2vWixy2W1WsfGv8g', 'EKEPtNKJdZxRhYoFhuG8ew', 'QMySP-qHdwQNZY0gDba0ng', '8PmbCVEjWGPouD1Tst6FsQ', 'E7s0QreUJ2H9r0Ls1yDeiQ', 'V0tRCjf95FTqVJvmxdeqbQ', 'z-3ZDmVbHZjDrYZE42yKZA', 'hz0ldhBHu5M4ri9M_mkOdQ', 'jXigX_-U5IZst165QXWExg', 'FMoKoSHgitwnt_iMWm31nQ']
The result for user 1 is: ['adI5BlJCtvWq2uAwmyndXA', 'QMySP-qHdwQNZY0gDba0ng', 'hJNjKc4sKgWG__KitlJ1ag', 'aIs7Tt3pw6ymj3FOPPR4Ng', '5Iin0tl6QJT_TyL1BldtyA', 'sxuEde0_sVFY-Uk9gpHy_g', '8Haoz9Mil7ZnHGU48sgBtw', 'qesu3mk5yHeak9CUO4gRdQ', '7Oi6eT_-h_9JlMqRsEdt3A', 'Sqwa2aNwnqCmc95gVWQ9bA']
The result for user 2 is: ['Q0fcX_1wvdmffqEPa246rg', 'xWuzWH8qYaa75REqVd9Isw', 'J_KI8rJa_dSwTglUbwFcdw', 'xbCNlp2vWixy2W1WsfGv8g', 'I1j1kEbQ5BliKsjzl5etVA', 'KL1oif2ocGH2dJ4w-oT_Mg', 'BmrVL3Aj5usPfpu5DE1iCg', '563C7-HcjhZwZSl4fK-mfQ', 'nbJXmGpdCxiBnovBkBtWcA', 'qeFGUu1WPvMMNn5Hno6D-Q']
The result for user 3 is: ['Q0fcX_1wvdmffqEPa246rg', 'ZOyBVpeSQ7sYDeZa_v0MFw', 'lWRbdK9YABr0auLG19FHhQ', 'I1j1kEbQ5BliKsjzl5etVA', '6MR9GZj

SỬ DỤNG LLM ĐỂ RE-RANK

In [34]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


In [46]:
def re_rank_candidates(user_id, candidate_restaurants, user_keywords):
    """
    Sử dụng GPT-4o API để re-rank các candidate restaurants dựa trên từ khóa của người dùng.

    Args:
        user_id (str): Mã định danh của người dùng.
        candidate_restaurants (list): Danh sách tên restaurant được candidate.
        user_keywords (list): Danh sách từ khóa liên quan đến sở thích của người dùng.

    Returns:
        list: Danh sách restaurant được xếp hạng lại.
    """
    prompt = (
        f"Người dùng có sở thích: {', '.join(user_keywords[:5])}. " 
        f"Với các candidate restaurants sau: {', '.join(candidate_restaurants[:5])}. "  
        "Hãy xếp hạng lại các restaurant theo mức độ phù hợp với sở thích của người dùng theo thứ tự giảm dần và chỉ in ra danh sách tên các restaurant, cách nhau bằng dấu phẩy."
    )

    response = openai.Completion.create(
        model="gpt-4o",  
        prompt=prompt,
        max_tokens=200,  
        temperature=0.7,  
        n=1,
        stop=["\n"]
    )

    generated = response.choices[0].text.strip()

    re_ranked = [restaurant.strip() for restaurant in generated.split(',')]

    valid_re_ranked = [r for r in re_ranked if r in candidate_restaurants]
    if len(valid_re_ranked) == 0:
        valid_re_ranked = candidate_restaurants
    return valid_re_ranked

final_results = []
for idx, (user, candidate_indices) in enumerate(zip(test_users, results)):
    candidate_restaurants = [restaurant_set[i] for i in candidate_indices]
    user_kw = test_users2kw[idx]
    re_ranked = re_rank_candidates(user, candidate_restaurants, user_kw)
    final_results.append(re_ranked)
    print(f"Re-ranked results for user {user}: {re_ranked}")

APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [18]:
llm_pipeline = pipeline("text-generation", model="gpt2")

def re_rank_candidates(user_id, candidate_restaurants, user_keywords):
    """
    Sử dụng LLM để re-rank các candidate restaurants dựa trên từ khóa của người dùng.

    Args:
        user_id (str): Mã định danh của người dùng.
        candidate_restaurants (list): Danh sách tên restaurant được candidate.
        user_keywords (list): Danh sách từ khóa liên quan đến sở thích của người dùng.

    Returns:
        list: Danh sách restaurant được xếp hạng lại.
    """
    prompt = (
        f"Người dùng có sở thích: {', '.join(user_keywords[:5])}. "  
        f"Với các candidate restaurants sau: {', '.join(candidate_restaurants[:5])}. "  
        "Hãy xếp hạng lại các restaurant theo mức độ phù hợp với sở thích của người dùng theo thứ tự giảm dần và chỉ in ra danh sách tên các restaurant, cách nhau bằng dấu phẩy."
    )
    
    generated = llm_pipeline(prompt, max_new_tokens=200, do_sample=False)[0]['generated_text']
    
    output = generated[len(prompt):].strip()
    
    re_ranked = [restaurant.strip() for restaurant in output.split(',')]
    
    valid_re_ranked = [r for r in re_ranked if r in candidate_restaurants]
    if len(valid_re_ranked) == 0:
        valid_re_ranked = candidate_restaurants
    return valid_re_ranked

final_results = []
for idx, (user, candidate_indices) in enumerate(zip(test_users, results)):
    candidate_restaurants = [restaurant_set[i] for i in candidate_indices]
    user_kw = test_users2kw[idx]
    re_ranked = re_rank_candidates(user, candidate_restaurants, user_kw)
    final_results.append(re_ranked)
    print(f"Re-ranked results for user {user}: {re_ranked}")

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Re-ranked results for user yfXqZkU5iXE07GSHzdsQBA: ['xbCNlp2vWixy2W1WsfGv8g', 'EKEPtNKJdZxRhYoFhuG8ew', 'QMySP-qHdwQNZY0gDba0ng', '8PmbCVEjWGPouD1Tst6FsQ', 'E7s0QreUJ2H9r0Ls1yDeiQ', 'V0tRCjf95FTqVJvmxdeqbQ', 'z-3ZDmVbHZjDrYZE42yKZA', 'hz0ldhBHu5M4ri9M_mkOdQ', 'jXigX_-U5IZst165QXWExg', 'FMoKoSHgitwnt_iMWm31nQ']


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Re-ranked results for user hutJzKEYHuVq6CP-XSARgg: ['adI5BlJCtvWq2uAwmyndXA', 'QMySP-qHdwQNZY0gDba0ng', 'hJNjKc4sKgWG__KitlJ1ag', 'aIs7Tt3pw6ymj3FOPPR4Ng', '5Iin0tl6QJT_TyL1BldtyA', 'sxuEde0_sVFY-Uk9gpHy_g', '8Haoz9Mil7ZnHGU48sgBtw', 'qesu3mk5yHeak9CUO4gRdQ', '7Oi6eT_-h_9JlMqRsEdt3A', 'Sqwa2aNwnqCmc95gVWQ9bA']


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Re-ranked results for user nr3S8p-lpf7K9W58eo_mLQ: ['mVZoin0a8uTZgLsgJ9iNDQ', '-JPz4W7MPOy-rQO8OKNk0g', '8PmbCVEjWGPouD1Tst6FsQ', 'APN05ziH92oIox8EMxcceg', 'nbJXmGpdCxiBnovBkBtWcA', '6MR9GZjWjLAS3s39YdDXKw', 'Q0fcX_1wvdmffqEPa246rg', 'KL1oif2ocGH2dJ4w-oT_Mg', 'WUAe0FGaLRCremEDXqNV4A', 'r_KDxVcTZVEigYepnoJ6Lw']


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Re-ranked results for user K1tDoNEHiaAi-k4SFhCAng: ['gONO6UIMj8kFo1IvnROuuQ', 'S1eha-aT5c7WMk7lDLsW0A', '563C7-HcjhZwZSl4fK-mfQ', '2HMvoTz_hw0noLzBWmyxcA', 'xWuzWH8qYaa75REqVd9Isw', 'krvqEVmPnJJYA9ngZCHX2w', 'vrEwYWESbL5fDEzdZacJnA', 'hz0ldhBHu5M4ri9M_mkOdQ', 'Sqwa2aNwnqCmc95gVWQ9bA', 'sH-11zOhWubXobEeuMo2gg']


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Re-ranked results for user 0VyDSM1uIrTU8LfaFh8JdA: ['BXeXS9EaNN03AptFwElvjA', 'PiT0jyLL2Jwm2ODTDapOcg', 'nJdNzQ5Z278gTCFJ4HQBLA', 'S9XKKA1evhRh16zrXpSF7g', 'csbBCysrsifg8N10c0eHCA', 'z57omnqRcAODGwbwMHEIyw', 'LUhVj-r2TtmFx6cyF9ilJg', 'sH-11zOhWubXobEeuMo2gg', 'S1eha-aT5c7WMk7lDLsW0A', 'hJNjKc4sKgWG__KitlJ1ag']
Re-ranked results for user TNxBN-tg_2U0vBnTw-LZKQ: ['563C7-HcjhZwZSl4fK-mfQ', 'nJdNzQ5Z278gTCFJ4HQBLA', '6Kl2SQp-zGo-nsdImLykxw', 'Rx85NUdD6MhrgKxHr05NlQ', 'MuVIpRAEkNfRnmhsPTdl3A', 'qDPn14lenAGehkbnJaqnYg', 'lrPNLw0zbfzgWrWINMjsNw', 'e8ohWYW8rbOZWZ4pojU4zQ', 'xjBvbDmbsA0VX-vA55SbAA', '8PmbCVEjWGPouD1Tst6FsQ']


In [ ]:
import json

output_data = {}

for idx, (user, restaurant_indices) in enumerate(zip(test_users, results)):
    user_data = {}
    
    user_keywords = test_users2kw[idx]
    
    candidate_restaurants = [restaurant_set[i] for i in restaurant_indices]
    
    positions = [str(i) for i in restaurant_indices]

    re_ranked = re_rank_candidates(user, candidate_restaurants, user_keywords)
    
    user_data["kw"] = user_keywords[:10]
    user_data["candidate"] = re_ranked[:10]
    user_data["positions"] = positions[:10]
    
    output_data[user] = user_data

json_file_path = './result/results.json'

with open(json_file_path, mode="w", encoding="utf-8") as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4)

print(f"\nResults saved to: {json_file_path}")



Results saved to: ./result/results.json


In [23]:
import json


output_data = {}

for idx, (user, restaurant_indices) in enumerate(zip(test_users, results)):
    user_data = {}

    user_keywords = test_users2kw[idx]

    candidate_restaurants = [restaurant_set[i] for i in restaurant_indices]

    re_ranked_restaurants = re_ranked

    positions = [str(i) for i in restaurant_indices]  # 1-based index

    user_data["kw"] = user_keywords[:10]  
    user_data["candidate"] = re_ranked_restaurants[:10] 
    user_data["positions"] = positions[:10]  
    output_data[user] = user_data

file_path='./result/results(afterLLM).json'

with open(file_path, mode="w", encoding="utf-8") as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4)

print(f"Results saved to: {file_path}")


Results saved to: ./result/results(afterLLM).json
